## Jupyter Notebook para desarrollar la Task 2

Lectura de los DataFrames:

In [1]:
from pathlib import Path    
import pandas as pd

RAW_DATA_PATH = Path('../../data/raw')

train_df = pd.read_csv(         RAW_DATA_PATH / 'train.csv')
test_df = pd.read_csv(          RAW_DATA_PATH / 'test.csv')
products_df = pd.read_pickle(   RAW_DATA_PATH / 'products.pkl')
users_df = pd.read_csv(         RAW_DATA_PATH / 'users_data.csv')

In [2]:
import ast

# Extraer datos del string (JSON) de la columna 'values'
def extract_data_from_string(df, column_name):
    
    df[column_name] = df[column_name].apply(ast.literal_eval)
    
    df['country'] = df[column_name].apply(lambda x: x['country'])
    df['R'] = df[column_name].apply(lambda x: x['R'])
    df['F'] = df[column_name].apply(lambda x: x['F'])
    df['M'] = df[column_name].apply(lambda x: x['M'])
    
    df = df.drop(columns=[column_name])
    
    df = df.explode(['country', 'R', 'F', 'M'])
    
    df['country'] = df['country'].astype(int)
    df['R'] = df['R'].astype(int)
    df['F'] = df['F'].astype(int)
    df['M'] = df['M'].astype(float)
    
    return df

users_df = extract_data_from_string(users_df, 'values')

Pruebas:

In [27]:
import importlib
import session_metrics
importlib.reload(session_metrics) # Para obtener la version mas reciente cada vez que se ejecute
from session_metrics import get_session_metrics

a = get_session_metrics(train_df, user_id=8287)
print(a)

    user_id  session_id  total_session_time  cart_addition_ratio
0      8287      185012              122.25                20.00
1      8287      279302             1912.75                 0.00
2      8287      298726               58.92                 0.00
3      8287      323810               86.83                25.00
4      8287     1583804              199.17                 0.00
5      8287     1786356              173.03                 0.00
6      8287     1812557              105.79                 0.00
7      8287     2242336              242.41                 0.00
8      8287     2854366                0.00                 0.00
9      8287     2942009             1320.83                 0.00
10     8287     2978783              176.38                 0.00
11     8287     3028792                2.33                 0.00
12     8287     3564719              217.39                18.18
13     8287     3752200              513.27                 0.00
14     8287     3799661  

En la pregunta se dice:

    - cart_addition_ratio (float) : Percentage of the added products out of the total products interacted with. Rounded ot the 2nd decimal.

Tenemos el siguiente ejemplo:


In [25]:
# Filtramos los datos para el usuario
user_data_aux = train_df[train_df['user_id'] == 8287]
user_data_aux.loc[:,'timestamp_local'] = pd.to_datetime(user_data_aux['timestamp_local'])

session_data = user_data_aux[user_data_aux['session_id'] == 3564719]

# Calculamos el tiempo total de la sesión
total_session_time_aux = (session_data['timestamp_local'].max() - session_data['timestamp_local'].min()).total_seconds()
total_session_time_aux = round(total_session_time_aux, 2)

# Calculamos la proporción de productos añadidos al carrito
total_products_cart_aux = session_data[session_data['add_to_cart']==1]['partnumber'].nunique()
total_products_visited_aux = session_data['partnumber'].nunique()
cart_addition_ratio_aux = round(100 * total_products_cart_aux / total_products_visited_aux if total_products_visited_aux > 0 else 0, 2)

a = session_data.groupby('partnumber').agg({'add_to_cart': ['sum','count']}).reset_index()

print(a)


  partnumber add_to_cart      
                     sum count
0       6703           1     2
1       7348           0     1
2      11145           1     2
3      11493           0     1
4      19515           0     1
5      30740           0     3
6      40810           0     1


- Hay 7 productos, 2 han sido agregados al carrito, lo que nos da 28.57% de productos añadidos al carro sobre el total de productos con los que se ha interaccionado siendo estrictos con la pregunta.
- Ahora bien, si calculamos la proporción de adiciones al carrito sobre el numero total de visitas , nos da 18.18% que es lo que devuelven los tests unitarios...